# About this notebook

- PyTorch NN starter code
- MultilabelStratifiedKFold 5 folds 

If this notebook is helpful, feel free to upvote :)

# Library

In [1]:
#import sys
#sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import os
import gc
import random
import math
import time
from collections import defaultdict

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss

import category_encoders as ce

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import common as com
from MLP_Dataset import TrainDataset, TestDataset
from MLP_Model import FCBlock, TabularNN
from trainer import train_fn, validate_fn, inference_fn, AverageMeter
from config import CFG

# Utils

In [3]:
out_base_path = "./result"
model_path = out_base_path + '/model'
os.makedirs(out_base_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

In [4]:
logger = com.get_logger(out_base_path+'/mlp_baseline')
com.seed_everything(seed=42)

# Data Loading

In [5]:
base_dir = '/media/hiroki/working/kaggle/Mechanisms-of-Action-Prediction/datasets'
#os.listdir('../input/lish-moa')
os.listdir(base_dir)

['sample_submission.csv',
 'test_features.csv',
 'train_features.csv',
 'train_targets_nonscored.csv',
 'train_targets_scored.csv']

In [6]:
train_features = pd.read_csv(base_dir + '/train_features.csv')
train_targets_scored = pd.read_csv(base_dir + '/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(base_dir + '/train_targets_nonscored.csv')
test_features = pd.read_csv(base_dir + '/test_features.csv')
submission = pd.read_csv(base_dir + '/sample_submission.csv')

#train_features = pd.read_csv('../input/lish-moa/train_features.csv')
#train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
#train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
#test_features = pd.read_csv('../input/lish-moa/test_features.csv')
#submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [7]:
# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
train = train_features.merge(train_targets_scored, on='sig_id')
target_cols = [c for c in train_targets_scored.columns if c not in ['sig_id']]
cols = target_cols + ['cp_type']
train[cols].groupby('cp_type').sum().sum(1)

cp_type
ctl_vehicle        0
trt_cp         16844
dtype: int64

- labels for 'ctl_vehicle' are all 0.

In [8]:
# constrcut train&test except 'cp_type'=='ctl_vehicle' data
print(train_features.shape, test_features.shape)
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
print(train.shape, test.shape)

(23814, 876) (3982, 876)
(21948, 1082) (3624, 876)


# CV split

In [9]:
folds = train.copy()
Fold = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[target_cols])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.shape)

(21948, 1083)


# Dataset

In [10]:
cat_features = ['cp_time', 'cp_dose']
num_features = [c for c in train.columns if train.dtypes[c] != 'object']
num_features = [c for c in num_features if c not in cat_features]
num_features = [c for c in num_features if c not in target_cols]
target = train[target_cols].values

def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 3, 'D2': 4})
    return df

train = cate2num(train)
test = cate2num(test)

# MODEL

In [11]:
def run_single_nn(cfg, train, test, folds, num_features, cat_features, target, target_cols, device, model_path, fold_num=0, seed=42):
    
    # Set seed
    logger.info(f'Set seed {seed}')
    com.seed_everything(seed=seed)

    # loader
    trn_idx = folds[folds['fold'] != fold_num].index
    val_idx = folds[folds['fold'] == fold_num].index
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    train_target = target[trn_idx]
    valid_target = target[val_idx]
    train_dataset = TrainDataset(train_folds, num_features, cat_features, train_target)
    valid_dataset = TrainDataset(valid_folds, num_features, cat_features, valid_target)
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, 
                              num_workers=4, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, 
                              num_workers=4, pin_memory=True, drop_last=False)

    # model
    model = TabularNN(cfg, num_features, target_cols)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=cfg.epochs, steps_per_epoch=len(train_loader))

    # log
    log_df = pd.DataFrame(columns=(['EPOCH']+['TRAIN_LOSS']+['VALID_LOSS']) )

    # train & validate
    best_loss = np.inf
    early_stopping_cnt = 0
    for epoch in range(cfg.epochs):
        train_loss = train_fn(train_loader, model, optimizer, epoch, scheduler, device)
        valid_loss, val_preds = validate_fn(valid_loader, model, device)
        log_row = {'EPOCH': epoch, 
                   'TRAIN_LOSS': train_loss,
                   'VALID_LOSS': valid_loss,
                  }
        log_df = log_df.append(pd.DataFrame(log_row, index=[0]), sort=False)
        #logger.info(log_df.tail(1))
        if valid_loss < best_loss:
            logger.info(f'epoch{epoch} save best model... tr_loss:{train_loss}, val_loss{valid_loss}')
            best_loss = valid_loss
            oof = np.zeros((len(train), len(target_cols)))
            oof[val_idx] = val_preds
            torch.save(model.state_dict(), model_path + f"/fold{fold_num}_seed{seed}.pth")
            best_epoch = epoch
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
            if early_stopping_cnt == cfg.early_stopping_rounds:
                logger.info(f'best epoch: epoch{best_epoch}')
                break

    # predictions
    test_dataset = TestDataset(test, num_features, cat_features)
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
                             num_workers=4, pin_memory=True)
    model = TabularNN(cfg, num_features, target_cols)
    model.load_state_dict(torch.load(model_path + f"/fold{fold_num}_seed{seed}.pth"))
    model.to(device)
    predictions = inference_fn(test_loader, model, device)
    
    # del
    torch.cuda.empty_cache()

    return oof, predictions, log_df


def run_kfold_nn(cfg, train, test, folds, num_features, cat_features, target, target_cols, device, model_path, n_fold=5, seed=42):

    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    log_dfs = []
    for _fold in range(n_fold):
        logger.info("Fold {}".format(_fold))
        _oof, _predictions, log_df = run_single_nn(cfg,
                                                   train,
                                                   test,
                                                   folds,
                                                   num_features, 
                                                   cat_features,
                                                   target,
                                                   target_cols,
                                                   device,
                                                   model_path,
                                                   fold_num=_fold,
                                                   seed=seed)
        oof += _oof
        predictions += _predictions / n_fold
        log_dfs.append(log_df)

    score = 0
    for i in range(target.shape[1]):
        _score = log_loss(target[:,i], oof[:,i])
        score += _score / target.shape[1]
    logger.info(f"CV score: {score}")
    
    return oof, predictions, log_dfs

In [12]:
# Seed Averaging for solid result
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

#SEED = [0, 1, 2]
#for seed in SEED:
oof, predictions, log_dfs = run_kfold_nn(CFG,
                                         train, test, folds, 
                                         num_features, cat_features, target, target_cols,
                                         device,
                                         model_path=model_path,
                                         n_fold=5, seed=42)
score = 0
for i in range(target.shape[1]):
    _score = log_loss(target[:,i], oof[:,i])
    score += _score / target.shape[1]
#logger.info(f"Seed Averaged CV score: {score}")

Fold 0
Set seed 42
epoch0 save best model... tr_loss:0.572882821361949, val_loss0.5055522397058699
epoch1 save best model... tr_loss:0.342186775031316, val_loss0.3608704112113744
epoch2 save best model... tr_loss:0.21028415265962155, val_loss0.26862836090470227
epoch3 save best model... tr_loss:0.13687151915183032, val_loss0.20710390886304592
epoch4 save best model... tr_loss:0.09411746970493429, val_loss0.16338148814942136
epoch5 save best model... tr_loss:0.06857267672019283, val_loss0.12882312998209564
epoch6 save best model... tr_loss:0.05224197517645403, val_loss0.10252953714824754
epoch7 save best model... tr_loss:0.04165798866863016, val_loss0.08503408500911978
epoch8 save best model... tr_loss:0.03485181750253822, val_loss0.07168381131319902
epoch9 save best model... tr_loss:0.030345757138391917, val_loss0.05983186124024196
epoch10 save best model... tr_loss:0.027196080542856108, val_loss0.052748787168897096
epoch11 save best model... tr_loss:0.025060093009939593, val_loss0.046

epoch59 save best model... tr_loss:0.013093302247325217, val_loss0.017706216359192797
epoch61 save best model... tr_loss:0.0127161058918471, val_loss0.017621068983149964
epoch62 save best model... tr_loss:0.012558189636252711, val_loss0.01744191476434537
epoch65 save best model... tr_loss:0.012091419843100283, val_loss0.01742034893745985
epoch67 save best model... tr_loss:0.011891623593554118, val_loss0.01741029226511785
epoch68 save best model... tr_loss:0.011802691526126362, val_loss0.017372406695827264
best epoch: epoch68
Fold 2
Set seed 42
epoch0 save best model... tr_loss:0.5722914976577689, val_loss0.5030687253291581
epoch1 save best model... tr_loss:0.34227882369156304, val_loss0.36360373964720366
epoch2 save best model... tr_loss:0.21077583102087905, val_loss0.2703187836414319
epoch3 save best model... tr_loss:0.13707466964630316, val_loss0.20517804420165467
epoch4 save best model... tr_loss:0.09435263261144614, val_loss0.15963667210494503
epoch5 save best model... tr_loss:0.06

epoch37 save best model... tr_loss:0.016526786809324893, val_loss0.01981217053916161
epoch39 save best model... tr_loss:0.016240667767591612, val_loss0.019454394161684232
epoch41 save best model... tr_loss:0.01590117377256227, val_loss0.019090576920072825
epoch44 save best model... tr_loss:0.015486555077971713, val_loss0.018943852161594023
epoch45 save best model... tr_loss:0.0152999267269293, val_loss0.01874624118208885
epoch47 save best model... tr_loss:0.01503147807543295, val_loss0.018539549822331424
epoch49 save best model... tr_loss:0.014720801542764598, val_loss0.01819260692890046
epoch50 save best model... tr_loss:0.014571761093816183, val_loss0.017983584013048365
epoch54 save best model... tr_loss:0.013956892484268785, val_loss0.017842890133398133
epoch55 save best model... tr_loss:0.013794450704552179, val_loss0.017744882870677662
epoch56 save best model... tr_loss:0.013652226580034968, val_loss0.01773767105318555
epoch57 save best model... tr_loss:0.013493758752044753, val_l

In [13]:
train[target_cols] = oof
train[['sig_id']+target_cols].to_csv(out_base_path + '/oof.csv', index=False)

test[target_cols] = predictions
test[['sig_id']+target_cols].to_csv(out_base_path + '/pred.csv', index=False)

In [14]:
# Final result with 'cp_type'=='ctl_vehicle' data
result = train_targets_scored.drop(columns=target_cols)\
            .merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
y_true = train_targets_scored[target_cols].values
y_pred = result[target_cols].values
score = 0
for i in range(y_true.shape[1]):
    _score = log_loss(y_true[:,i], y_pred[:,i])
    score += _score / y_true.shape[1]
logger.info(f"Final result: {score}")

Final result: 0.015842093180158456


# eval

In [ ]:
test[target_cols] = predictions
test[['sig_id']+target_cols].to_csv('pred.csv', index=False)

# Submit

In [ ]:
sub = submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)
sub.head()